In [134]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.express as px
import cv2
from PIL import Image

Định dạng kiểu vẽ đồ thị là seaborn và thiết lập kích thước của các đồ thị

In [135]:
plt.style.use("seaborn")
plt.rcParams['figure.figsize'] = (13, 7)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_36732\4225139297.py:1: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


# Load data


In [136]:
data_df = pd.read_csv(r"C:\Users\ADMIN\OneDrive - Hanoi University of Science and Technology\2022-2\Hệ hỗ trợ quyết định\archive\book_data.csv")    

In [137]:
data_df.head()

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
0,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075.0,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/5e/18/24/2...
1,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075.0,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/5e/18/24/2...
2,184466860,Hành Tinh Của Một Kẻ Nghĩ Nhiều,Nguyễn Đoàn Minh Thư,86000,59900,7929.0,Sách tư duy - Kỹ năng sống,780,4.8,184,Nhà Xuất Bản Thế Giới,https://salt.tikicdn.com/ts/product/d7/99/24/1...
3,73787185,Những Tù Nhân Của Địa Lý,Tim Marshall,210000,126000,17896.0,Lĩnh vực khác,3623,4.8,430,Nhà Xuất Bản Hội Nhà Văn,https://salt.tikicdn.com/ts/product/8d/96/9e/c...
4,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,79000,47400,24668.0,Tác phẩm kinh điển,5131,5.0,228,Nhà Xuất Bản Hà Nội,https://salt.tikicdn.com/ts/product/45/3b/fc/a...


In [138]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796 entries, 0 to 1795
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      1796 non-null   int64  
 1   title           1796 non-null   object 
 2   authors         1653 non-null   object 
 3   original_price  1796 non-null   int64  
 4   current_price   1796 non-null   int64  
 5   quantity        1751 non-null   float64
 6   category        1796 non-null   object 
 7   n_review        1796 non-null   int64  
 8   avg_rating      1796 non-null   float64
 9   pages           1546 non-null   object 
 10  manufacturer    1523 non-null   object 
 11  cover_link      1796 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 168.5+ KB


# Data preparation

Kiểm tra xem dữ liệu từng cột có bao nhiêu giá trị null

In [139]:
data_df.isnull().sum()

product_id          0
title               0
authors           143
original_price      0
current_price       0
quantity           45
category            0
n_review            0
avg_rating          0
pages             250
manufacturer      273
cover_link          0
dtype: int64

## Title

Xóa các dữ liệu có cùng tiêu đề. 

In [140]:
data_df = data_df.drop_duplicates(subset=['title'])


## Authors

Số lượng sách của các tác giả có trong cơ sở dữ liệu

In [141]:
data_df.authors.value_counts()

Nguyễn Nhật Ánh    24
Higashino Keigo    20
.                  18
Thích Nhất Hạnh    16
Haruki Murakami    15
                   ..
Urako Kanamori      1
Cổ Viên             1
Robert Winston      1
Yongchul Kwon       1
John C. Maxwell     1
Name: authors, Length: 1083, dtype: int64

Có tới 18 cuốn sách có tác giả là . -> ta sẽ điền tên tác giả này là "Unknown"


In [142]:
data_df.loc[data_df.authors == '.' , 'authors'] = "Unknown"

In [143]:
data_df.authors.value_counts()

Nguyễn Nhật Ánh    24
Higashino Keigo    20
Unknown            18
Thích Nhất Hạnh    16
Haruki Murakami    15
                   ..
Urako Kanamori      1
Cổ Viên             1
Robert Winston      1
Yongchul Kwon       1
John C. Maxwell     1
Name: authors, Length: 1083, dtype: int64

In [144]:
data_df.authors.isnull().sum()


138

Cột authors có tới 138 giá trị null, ta sẽ cũng sẽ đặt là "Unknown" cho các bản ghi này. '

In [145]:
data_df.authors = data_df.authors.fillna("Unknown")
data_df.authors.value_counts()

Unknown            156
Nguyễn Nhật Ánh     24
Higashino Keigo     20
Thích Nhất Hạnh     16
Haruki Murakami     15
                  ... 
Urako Kanamori       1
Cổ Viên              1
Robert Winston       1
Yongchul Kwon        1
John C. Maxwell      1
Name: authors, Length: 1083, dtype: int64

## Quantity

In [146]:
data_df.quantity.isnull().sum()

41

Có 41 cuốn sách không có dữ liệu về doanh số, ta sẽ drop các bản ghi này

In [147]:
data_df.dropna(subset=['quantity'], inplace=True)


## Category

In [148]:
data_df.category.value_counts()


Sách tư duy - Kỹ năng sống                 280
Tiểu Thuyết                                127
Truyện ngắn - Tản văn - Tạp Văn            109
Sách nghệ thuật sống đẹp                    57
Sách kỹ năng làm việc                       53
                                          ... 
Shaman King - Tập 19                         1
Blue Period - Tập 07                         1
Komi - Nữ Thần Sợ Giao Tiếp - Tập 14         1
Bên Rặng Tuyết Sơn (Tái Bản)                 1
Kaguya-Sama: Cuộc Chiến Tỏ Tình - Tập 3      1
Name: category, Length: 350, dtype: int64

In [149]:
sum_category1 = np.where(data_df.category.value_counts() > 2, 1, 0)
sum_category1.sum()

61

Có nhiều thể loại có thể viết sai trong quá trinh thu thập dữ liệu \
Ta sẽ chỉ lấy 61 thể loại đầu tiên, còn lại ta sẽ đặt là "Others"

In [150]:
keeping_values = list(pd.DataFrame(data_df.category.value_counts()[1:61]).T.columns)
keeping_values

['Tiểu Thuyết',
 'Truyện ngắn - Tản văn - Tạp Văn',
 'Sách nghệ thuật sống đẹp',
 'Sách kỹ năng làm việc',
 'Bài học kinh doanh',
 'Truyện trinh thám',
 'Sách Học Tiếng Anh',
 'Sách Marketing - Bán hàng ',
 'Tác phẩm kinh điển',
 'Truyện Giả tưởng - Huyền Bí - Phiêu Lưu',
 'Sách tài chính, tiền tệ',
 'Kiến thức - Bách khoa',
 'Sách Làm Cha Mẹ',
 'Lĩnh vực khác',
 'Truyện dài',
 'Kiến Thức Bách Khoa',
 'Văn học thiếu nhi',
 'Light novel',
 'Truyện đam mỹ',
 'Truyện kể cho bé',
 'Sách quản trị, lãnh đạo',
 'Tiểu sử - Hồi ký',
 'Lịch Sử Thế Giới',
 'Sách Chiêm Tinh - Horoscope',
 'Truyện tranh Ehon',
 'Sách Học Tiếng Hoa',
 'Sách kinh tế học',
 'Sách giáo dục',
 'Tô màu - Luyện chữ  ',
 'Thơ',
 'Truyện tranh thiếu nhi',
 'Sách khởi nghiệp',
 'Triết Học',
 'Sách Nấu ăn',
 'Sách Luyện Thi Đại Học - Cao Đẳng',
 'Lịch Sử Việt Nam',
 'Du ký',
 'Sách Tâm Lý Tuổi Teen',
 'Mỹ Thuật - Kiến Trúc',
 'Truyện ngôn tình',
 'Sách Hôn Nhân - Giới Tính',
 'Truyện kinh dị',
 'Tranh Truyện',
 'Sách doanh nh

In [151]:
def handle_category(category):
    if category not in keeping_values:
        return "Others"
    return category

In [152]:
data_df.category = data_df.category.apply(lambda category: handle_category(category))

In [153]:
data_df.category.value_counts()

Others                             577
Tiểu Thuyết                        127
Truyện ngắn - Tản văn - Tạp Văn    109
Sách nghệ thuật sống đẹp            57
Bài học kinh doanh                  53
                                  ... 
Đạo đức - Kỹ năng sống               3
Sách Tô Màu Dành Cho Người Lớn       3
Truyện cười                          3
Chăm sóc sức khỏe                    3
Sách tham khảo cấp III               3
Name: category, Length: 61, dtype: int64

## Page


In [154]:
data_df.pages.isnull().sum()

232

Có tới 232 cuốn sách bị thiếu dữ liệu về số trang. Tiếp theo ta sẽ điền số trang của còn thiếu là giá trị trung bình. 

In [155]:
data_df.query('pages == "Cuốn"')


,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
1266,197639028,Động Từ Bất Quy Tắc Và Ngữ Pháp Tiếng Anh Căn Bản,Mai Lan Hương,18000,12000,136.0,Sách Học Tiếng Anh,2,5.0,Cuốn,Nhà Xuất Bản Đà Nẵng,https://salt.tikicdn.com/ts/product/28/8e/9e/0...


In [158]:
data_df = data_df.drop(data_df[data_df['pages'] == 'Cuốn'].index)

In [159]:
data_df.query('pages == "Cuốn"')


,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link


In [161]:
data_df.query('pages == "192000"')



,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link
650,201687274,Có Một Nổi Buồn Vừa Ngang Qua Đây,Gã,88000,64900,114.0,Truyện ngắn - Tản văn - Tạp Văn,6,5.0,192000,Nhà Xuất Bản Văn Học,https://salt.tikicdn.com/ts/product/08/fd/a5/0...


In [164]:
data_df = data_df.drop(data_df[data_df['pages'] == '192000'].index)

In [165]:
data_df.query('pages == "192000"')


,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer,cover_link


In [166]:
def handle_pages(lst_pages):
    lst_int = []
    for page in lst_pages:
        try:
            lst_int.append(int(page))
        except:
            continue
            
    return np.mean(lst_int)

In [168]:
mean_pages = handle_pages(list(data_df.pages))
data_df.pages = data_df.pages.fillna(str(mean_pages))


In [169]:
data_df.pages = data_df.pages.apply(lambda page: round(float(page)))
data_df.pages = data_df.pages.astype("int64")

In [170]:
data_df.pages.describe()


count    1726.000000
mean      329.754345
std       175.509141
min        10.000000
25%       220.000000
50%       330.000000
75%       392.000000
max      2004.000000
Name: pages, dtype: float64

## Nhà xuất bản

In [173]:
data_df.manufacturer.value_counts()

Nhà Xuất Bản Thế Giới                            241
NXB Trẻ                                          143
Nhà Xuất Bản Hà Nội                              124
Nhà Xuất Bản Dân Trí                             121
Nhà Xuất Bản Kim Đồng                            111
Nhà Xuất Bản Hội Nhà Văn                         105
Nhà Xuất Bản Thanh Niên                           95
Nhà Xuất Bản Tổng hợp TP.HCM                      89
Nhà Xuất Bản Lao Động                             87
Nhà Xuất Bản Văn Học                              76
Nhà Xuất Bản Hồng Đức                             53
Nhà Xuất Bản Công Thương                          44
Nhà Xuất Bản Phụ Nữ Việt Nam                      38
Nhà Xuất Bản Phụ Nữ                               30
Nhà Xuất Bản Đà Nẵng                              16
Nhà Xuất Bản Đại Học Quốc Gia Hà Nội              12
Nhà Xuất Bản Mỹ Thuật                              8
Nhà Xuất Bản Tri Thức                              7
Nhà Xuất Bản Lao Động Xã Hội                  

Có một vài nhà xuất bản là cùng một nơi nhưng do đánh máy khác nhau nên sẽ trở thành 2 bản ghi khác nhau. \
Đó là: 
- Nhà Xuất Bản Hồng Đức & hong duc
- Nhà Xuất Bản Dân Trí & NXB Dân Trí
- Nhà Xuất Bản Đại Học Quốc Gia Hà Nội & ĐHQG Hà Nội


In [174]:
def handle_manufactuner(manufactuner):
    if manufactuner == "hong duc":
        return "Nhà Xuất Bản Hồng Đức"
    elif manufactuner == "NXB Dân Trí":
        return "Nhà Xuất Bản Dân Trí"
    elif manufactuner == "ĐHQG Hà Nội":
        return "Nhà Xuất Bản Đại Học Quốc Gia Hà Nội"
    else:
        return manufactuner
data_df.manufacturer = data_df.manufacturer.apply(lambda manufacturer: handle_manufactuner(manufacturer))


In [176]:
data_df.manufacturer.isnull().sum()

264

Có tới 264 bản ghi k chứa thông tin của nhà xuất bản, các giá trị này ta sẽ đặt là "Unknown". 

In [177]:
data_df.manufacturer = data_df.manufacturer.fillna("Unknown")

## Xử lý dữ liệu về giá bán. 


Tạo thêm cột mới chứa giá trị của giá đã giảm so với giá ban đầu. 

In [178]:
data_df['discount'] = round(1 - data_df['current_price']/data_df['original_price'],2)


In [179]:
data_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1726 entries, 0 to 1795
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      1726 non-null   int64  
 1   title           1726 non-null   object 
 2   authors         1726 non-null   object 
 3   original_price  1726 non-null   int64  
 4   current_price   1726 non-null   int64  
 5   quantity        1726 non-null   float64
 6   category        1726 non-null   object 
 7   n_review        1726 non-null   int64  
 8   avg_rating      1726 non-null   float64
 9   pages           1726 non-null   int64  
 10  manufacturer    1726 non-null   object 
 11  cover_link      1726 non-null   object 
 12  discount        1725 non-null   float64
dtypes: float64(3), int64(5), object(5)
memory usage: 188.8+ KB


In [186]:
data_df.title = data_df.title.astype("string")
data_df.authors = data_df.authors.astype("string")
data_df.category = data_df.category.astype("string")
data_df.manufacturer = data_df.manufacturer.astype("string")
data_df.cover_link = data_df.cover_link.astype("string")

data_df.quantity = data_df.quantity.apply(lambda quantity: round(float(quantity)))
data_df.quantity = data_df.quantity.astype("int64")

In [ ]:
data_df.to_csv("prepared_data_book.csv")
